In [12]:
from perfetto.trace_processor import TraceProcessor
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Initialise TraceProcessor with a trace file
color_list = ['blue','cyan','teal','red','pink','salmon']
line_list = ['-','-','-','-','-','-','-','-','-','-','-','-','-']
enable_legend = True



# exp=0
# phone_str = "Pixel7A"
# applications_list = ["5G_youtube_airplane","standby"] 
    
# tp_list = [TraceProcessor(trace="../Traces/"+phone_str+"/pixel7_"+application+"_1.perfetto-trace") 
#            for application in applications_list]

#  pixel7_sample_3mintrace.perfetto-trace

tp_list = [TraceProcessor(trace="../pixel7_sample_3mintrace.perfetto-trace") ]
tp_list.append(TraceProcessor(trace="../pixel7_sample_3mintrace_far.perfetto-trace"))
tp_list.append(TraceProcessor(trace="../pixel7_sample_3mintrace_gpt.perfetto-trace"))

num_tps = len(tp_list)

# qr_it_list = [tp.query('SELECT * FROM counter_track') for tp in tp_list]
# qr_it_list_pandas = [qr_it.as_pandas_dataframe() for qr_it in qr_it_list]

id_mapper = [14,15,2,1]
string_mapper = ["Modem","RFE","Wi-Fi","Display"]

df_list_cleaned = [[tp.query('SELECT ts,value FROM counter WHERE track_id='+str(i)+'')
                   .as_pandas_dataframe().replace(np.nan,0)
                   .values.tolist() for i in id_mapper] for tp in tp_list]

In [15]:
## plotly plots
color_list = ['blue','lightgreen','teal','pink','salmon']
# legend_list = applications_list
legend_list = ["FakeGPT","FakeGPT Far","ChatGPT app"]
# legend_list = ["Reading 1","Reading 2","Reading3","Standby_to_airplane","Standby"]
# legend_list = ["Standby to Wifi","Wifi Alone","Airplane alone","Standby"]

# string_mapper_AS = ["Cellular (Total)","Display"]
initial_offset = 2
# end_offset = 72
fig = make_subplots(rows=2,cols=2,subplot_titles=("Modem","RFE","Wi-Fi","Display"),column_widths=[1,1],row_heights=[1,1])

for tp_idx in range(num_tps):
    df_np = np.array(df_list_cleaned[tp_idx])
    df_np[:,:,0] = df_np[:,:,0] - np.tile(df_np[:,0,0],[df_np.shape[1],1]).T
#     df_np[:,:,:,0] = df_np[:,:,:,0] - np.tile(df_np[:,:,0,0],[df_np.shape[2],1,1]).transpose(1,2,0) 
    df_np[:,:,0] = df_np[:,:,0]/1e9
    
    for i in range(1,5):
        x_vals = df_np[i-1,:,0]
        y_vals = df_np[i-1,:,1]
#         x_vals = x_vals[:min(x_vals.size,end_offset)]
#         y_vals = y_vals[:min(y_vals.size,end_offset)]
        delta_y_vals = np.copy(y_vals)
        delta_y_vals[1:] = delta_y_vals[1:]-delta_y_vals[0:y_vals.size-1]
        delta_y_vals[0] = 0
        avg_pow_indiv = np.mean(delta_y_vals[initial_offset:]*(4/1000))
        print("Average Power (mW) for "+str(string_mapper[i-1])+": "+str(avg_pow_indiv)+" mW")
#         plt.subplot(2,2,i)
        fig.add_trace(go.Scatter(x=x_vals[initial_offset:],y=delta_y_vals[initial_offset:]*(4/1000),
                                  line_color= color_list[tp_idx],name=legend_list[tp_idx],
                                legendgroup=tp_idx,showlegend=(i==1)),row=int((i+1)/2),col=(i-1)%2+1)
        # *4 because power measurements done roughly over 250ms (1/4 seconds) interval
#         plt.plot(x_vals[initial_offset:],delta_y_vals[initial_offset:]*(4/1000), 
#                  color=color_list[tp_idx],linewidth=2, linestyle=line_list[tp_idx], label=legend_list[tp_idx])
        min_time = np.min(x_vals[initial_offset:])
        max_time = np.max(x_vals[initial_offset:])
        fig.add_trace(go.Scatter(x=[min_time,max_time],y=[avg_pow_indiv,avg_pow_indiv],
                                 mode='lines', line={'dash': 'dash', 'color': color_list[tp_idx]},name="Avg Pow of "+str(string_mapper[i-1])+": "+str(avg_pow_indiv.round(2))+ " mW",
                                 legendgroup=tp_idx),row=int((i+1)/2),col=(i-1)%2+1)
#         fig.add_annotation(xref="x domain", yref="y domain",
#                   x=1, y=1,text="Avg Pow: "+str(avg_pow_indiv.round(2))+ " mW",showarrow=True)
#         plt.xlabel("Time (seconds)", fontsize=12)
#         plt.ylabel("Power (mW)", fontsize=16)
#         plt.legend(fontsize=10)
        # plt.title('Ratio vs #femto BS')
#         plt.grid(visible=True,which='major',color='k', linestyle=':', linewidth=1, alpha=0.5)
#         plt.grid(visible=True,which='minor',color='k', linestyle=':', linewidth=1, alpha=0.1)
#         plt.minorticks_on()
        
#         plt.xticks(fontsize=12)
#         plt.yticks(fontsize=14)
fig.update_xaxes(title='Time')
fig.update_yaxes(range=[0,500],title='Power')
fig.update_layout(height=800, width=1.3*800, 
                  title=dict(text="Power consumption of UE with youtube and standby mode", font=dict(size=15)),
                  showlegend=enable_legend)

fig.show()
# fig_name = "random_sched"
# fig.write_html("./"+fig_name+".html")

Average Power (mW) for Modem: 501.48472120200336 mW
Average Power (mW) for RFE: 6.332166944908181 mW
Average Power (mW) for Wi-Fi: 21.142784641068452 mW
Average Power (mW) for Display: 171.40785976627714 mW
Average Power (mW) for Modem: 547.9221962421713 mW
Average Power (mW) for RFE: 6.870338204592903 mW
Average Power (mW) for Wi-Fi: 20.678797494780795 mW
Average Power (mW) for Display: 559.7424467640918 mW
Average Power (mW) for Modem: 460.1154924874791 mW
Average Power (mW) for RFE: 5.86289816360601 mW
Average Power (mW) for Wi-Fi: 23.58594323873122 mW
Average Power (mW) for Display: 135.79463772954924 mW
